In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/kaggle/src")

In [3]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold


NUM_BOOST_ROUND = 5000
STOPPING_ROUNDS = 20
VERBOSE_EVAL = 30


def quadratic_weighted_kappa(preds, data):
    y_true = data.get_label()
    y_pred = preds.clip(0, 3).round().astype(int)
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return "QWK", qwk, True


def threshold_rounder(oof_preds, thresholds):
    labels = []
    for value in oof_preds:
        if value < thresholds[0]:
            labels.append(0)
        elif value < thresholds[1]:
            labels.append(1)
        elif value < thresholds[2]:
            labels.append(2)
        else:
            labels.append(3)
    return np.array(labels)


def evaluate_predictions(thresholds, y_true, oof_preds):
    oof_preds_rounded = threshold_rounder(oof_preds, thresholds)
    return -cohen_kappa_score(y_true, oof_preds_rounded, weights="quadratic")


def train_kfold(features, targets, categorical_features, n_splits, model_params, weights=None):
    if weights is None:
        weights = np.ones(len(features))
    skf = StratifiedKFold(
        n_splits=n_splits, shuffle=True, random_state=1
    )
    boosters = []
    val_qwks = []
    oof_preds = np.zeros(len(features))

    for trn_idx, val_idx in skf.split(features, targets):
        trn_data = lgb.Dataset(
            data=features.iloc[trn_idx],
            label=targets.iloc[trn_idx],
            weight=weights[trn_idx],
            categorical_feature=categorical_features,
        )
        val_data = lgb.Dataset(
            data=features.iloc[val_idx],
            label=targets.iloc[val_idx],
            weight=weights[val_idx],
            reference=trn_data,
        )
        eval_result = {}
        callbacks = [
            lgb.early_stopping(STOPPING_ROUNDS),
            lgb.log_evaluation(VERBOSE_EVAL),
            lgb.record_evaluation(eval_result),
        ]
        booster = lgb.train(
            model_params,
            trn_data,
            valid_sets=[trn_data, val_data],
            valid_names=["trn", "val"],
            num_boost_round=NUM_BOOST_ROUND,
            feval=quadratic_weighted_kappa,
            callbacks=callbacks,
        )
        boosters.append(booster)
        val_qwks.append(
            eval_result["val"]["QWK"][booster.best_iteration - 1]
        )
        oof_preds[val_idx] = booster.predict(features.iloc[val_idx])
    print(f"Avg. Kappa: {np.mean(val_qwks):.4f}")
    return boosters, eval_result, val_qwks, oof_preds


def run_kfold(
        features, trn_targets, categorical_features, n_splits, model_params,
        weights=None, trn_id=None, tst_id=None
):
    trn_features = features.iloc[:len(trn_targets)]
    tst_features = features.iloc[len(trn_targets):]
    boosters, eval_result, qwks, oof_preds = train_kfold(
        trn_features, trn_targets, categorical_features, n_splits, model_params, weights
    )
    # save_loss
    loss = pd.DataFrame(
        data={
            "trn_rmse": eval_result["trn"]["rmse"],
            "val_rmse": eval_result["val"]["rmse"],
            "trn_qwk": eval_result["trn"]["QWK"],
            "val_qwk": eval_result["val"]["QWK"]
        }
    )
    print(loss)
    # kappa_optimizer
    kappa_optimizer = minimize(
        evaluate_predictions,
        x0=[0.5, 1.5, 2.5],
        args=(trn_targets.to_numpy(), oof_preds),
        method="Nelder-Mead",
    )
    # oof_preds
    oof_preds = threshold_rounder(oof_preds, kappa_optimizer.x)
    qwk_optimized = cohen_kappa_score(trn_targets.to_numpy(), oof_preds, weights="quadratic")
    print(f"Optimized Kappa: {qwk_optimized:.4f}")
    oof_preds = pd.DataFrame(
        data={"id": trn_id, "sii": oof_preds}
    )
    # tst_preds
    tst_preds = np.zeros(len(tst_features))
    for i, booster in enumerate(boosters):
        tst_preds += booster.predict(tst_features) / n_splits
    tst_preds = threshold_rounder(tst_preds, kappa_optimizer.x)
    tst_preds = pd.DataFrame(
        data={"id": tst_id, "sii": tst_preds}
    )
    tst_preds.to_csv("/kaggle/working/submission.csv", index=False)

In [4]:
# Import
import sys
sys.path.append("/kaggle/src")  # Change on kaggle notebook

from pathlib import Path

import polars as pl
from sklearn.preprocessing import LabelEncoder

from common.constant import TRAIN_CSV_PATH, TEST_CSV_PATH, OUTPUT_DIR
# from preprocess.table import fill_missing
# from run.lgb001 import run_kfold

# Data
trn_df = pl.read_csv(TRAIN_CSV_PATH)
tst_df = pl.read_csv(TEST_CSV_PATH)

base_cols = [col for col in trn_df.columns if col in tst_df.columns]
df = pl.concat([trn_df[base_cols], tst_df])

# Preprocessing
# df = fill_missing(df)

# Feature engineering
features = pl.concat(
    [
        df,
    ],
    how="horizontal",
)

trn_id = trn_df.drop_nulls(subset=["sii"])["id"].to_list()
tst_id = tst_df["id"].to_list()
features = pl.concat([
    features.slice(0, len(trn_df)).filter(pl.col("id").is_in(trn_id)),
    features.slice(len(trn_df)).filter(pl.col("id").is_in(tst_id))
])

# Label encoding
cat_cols = features.select(pl.col(pl.Utf8)).columns
cat_cols.remove("id")
le = LabelEncoder()
for col in cat_cols:
    encoded = le.fit_transform(features[col].to_numpy())
    features = features.with_columns(pl.Series(encoded).alias(col))

# Model parameters
model_params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting": "gbdt",
    "num_leaves": 64,
    "bagging_fraction": 0.8,
    "bagging_freq": 1,
    "feature_fraction": 0.8,
    "learning_rate": 0.1,
    "seed": 1,
    "num_threads": 4,
}



In [39]:
run_kfold(
    features=features.drop("id").to_pandas(),
    trn_targets=trn_df.filter(pl.col("id").is_in(trn_id))["sii"].to_pandas(),
    categorical_features=cat_cols,
    n_splits=10,
    model_params=model_params,
    weights=None,
    trn_id=trn_id,
    tst_id=tst_id,
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000681 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5987
[LightGBM] [Info] Number of data points in the train set: 2462, number of used features: 58
[LightGBM] [Info] Start training from score 0.580829
Training until validation scores don't improve for 20 rounds
[30]	trn's rmse: 0.429619	trn's QWK: 0.75525	val's rmse: 0.715368	val's QWK: 0.269592
Early stopping, best iteration is:
[14]	trn's rmse: 0.546517	trn's QWK: 0.594476	val's rmse: 0.699677	val's QWK: 0.311908
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5993
[LightGBM] [Info] Number of data points in the train set: 2462, number of used features: 58
[LightGBM] [Info] Start training from score 0.580829
Training until validation scores don't improve fo

In [5]:
import optuna

In [6]:
base_params = {
    "objective": "regression",
    "metric": "rmse",
    "boosting": "gbdt",
    "learning_rate": 0.01,
    "seed": 1,
    "num_threads": 4,
}

In [17]:
def train_kfold(features, targets, categorical_features, n_splits, base_params, weights=None):
    if weights is None:
        weights = np.ones(len(features))

    skf = StratifiedKFold(
        n_splits=n_splits, shuffle=True, random_state=1
    )
    boosters = []
    val_qwks = []
    oof_preds = np.zeros(len(features))

    for i, (trn_idx, val_idx) in enumerate(skf.split(features, targets)):
        trn_data = lgb.Dataset(
            data=features.iloc[trn_idx],
            label=targets.iloc[trn_idx],
            weight=weights[trn_idx],
            categorical_feature=categorical_features,
            free_raw_data=False
        )
        val_data = lgb.Dataset(
            data=features.iloc[val_idx],
            label=targets.iloc[val_idx],
            weight=weights[val_idx],
            categorical_feature=categorical_features,
            free_raw_data=False
        )

        if i == 0:          
            def objective(trial):
                params = {
                    # "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                    "max_depth": trial.suggest_int("max_depth", 1, 16),
                    "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 0, 300),
                    "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
                    "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                    "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
                    "min_child_samples": trial.suggest_int("min_child_sample", 5, 100),
                    "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
                    "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
                }
                eval_result = {}
                callbacks = [
                    lgb.early_stopping(STOPPING_ROUNDS),
                    # lgb.log_evaluation(VERBOSE_EVAL),
                    lgb.record_evaluation(eval_result),
                ]
                booster = lgb.train(
                    {**base_params, **params},
                    trn_data,
                    valid_sets=[trn_data, val_data],
                    valid_names=["trn", "val"],
                    num_boost_round=NUM_BOOST_ROUND,
                    callbacks=callbacks
                )
                return eval_result["val"]["rmse"][booster.best_iteration - 1]
            
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=100)
            best_params = study.best_params
            best_loss = study.best_value
            print("Best hyperparameters:", best_params)
            print("Best Loss:", best_loss)
        break   

In [8]:
features=features.drop("id").to_pandas()
trn_targets=trn_df.filter(pl.col("id").is_in(trn_id))["sii"].to_pandas()
trn_features = features.iloc[:len(trn_targets)]
tst_features = features.iloc[len(trn_targets):]

In [18]:
train_kfold(trn_features, trn_targets, categorical_features=cat_cols, n_splits=5, base_params=base_params)

[I 2024-10-27 21:27:07,769] A new study created in memory with name: no-name-74b4efd1-b076-4e83-b5b2-424e0473acb4
/tmp/ipykernel_390638/643600196.py:35: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
/tmp/ipykernel_390638/643600196.py:37: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
/tmp/ipykernel_390638/643600196.py:39: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "lambda_l1": trial.suggest_loguniform(

[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000656 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5928
[LightGBM] [Info] Number of data points in the train set: 2188, number of used features: 58
[LightGBM] [Warning] min_data_in_leaf is set=88, min_child_samples=47 will be ignored. Current value: min_data_in_leaf=88
[LightGBM] [Info] Start training from score 0.579982
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

[I 2024-10-27 21:27:07,986] Trial 1 finished with value: 0.6897554961878478 and parameters: {'num_leaves': 157, 'max_depth': 16, 'min_data_in_leaf': 171, 'bagging_fraction': 0.518254396949788, 'bagging_freq': 3, 'feature_fraction': 0.4376575663502093, 'min_child_sample': 49, 'lambda_l1': 4.182258497249038e-08, 'lambda_l2': 5.292711613203771e-08}. Best is trial 0 with value: 0.686093592307951.
[I 2024-10-27 21:27:08,070] Trial 2 finished with value: 0.6867137336256727 and parameters: {'num_leaves': 244, 'max_depth': 6, 'min_data_in_leaf': 133, 'bagging_fraction': 0.48531184859637566, 'bagging_freq': 7, 'feature_fraction': 0.41690189149731904, 'min_child_sample': 30, 'lambda_l1': 0.0005055790453529791, 'lambda_l2': 2.2283910029189652e-05}. Best is trial 0 with value: 0.686093592307951.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-10-27 21:27:08,223] Trial 3 finished with value: 0.6851443535799473 and parameters: {'num_leaves': 28, 'max_depth': 7, 'min_data_in_leaf': 240, 'bagging_fraction': 0.953021137183555, 'bagging_freq': 1, 'feature_fraction': 0.7454133921773181, 'min_child_sample': 85, 'lambda_l1': 7.020330891098437e-08, 'lambda_l2': 0.6249356332870412}. Best is trial 3 with value: 0.6851443535799473.
[I 2024-10-27 21:27:08,348] Trial 4 finished with value: 0.6825575741292037 and parameters: {'num_leaves': 117, 'max_depth': 13, 'min_data_in_leaf': 108, 'bagging_fraction': 0.8352529281032017, 'bagging_freq': 3, 'feature_fraction': 0.569625812459081, 'min_child_sample': 75, 'lambda_l1': 1.5970900400918077e-08, 'lambda_l2': 0.007610766621601987}. Best is trial 4 with value: 0.6825575741292037.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-10-27 21:27:24,872] Trial 98 finished with value: 0.682590893531199 and parameters: {'num_leaves': 247, 'max_depth': 8, 'min_data_in_leaf': 143, 'bagging_fraction': 0.8821861612192734, 'bagging_freq': 6, 'feature_fraction': 0.7111954869596907, 'min_child_sample': 89, 'lambda_l1': 0.0009425797441152771, 'lambda_l2': 0.1978033359931325}. Best is trial 39 with value: 0.6808455315610473.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-10-27 21:27:25,054] Trial 99 finished with value: 0.6838610010641184 and parameters: {'num_leaves': 236, 'max_depth': 8, 'min_data_in_leaf': 167, 'bagging_fraction': 0.9469621330130396, 'bagging_freq': 6, 'feature_fraction': 0.6659412358981854, 'min_child_sample': 18, 'lambda_l1': 0.0015600548747798087, 'lambda_l2': 0.39137277357064343}. Best is trial 39 with value: 0.6808455315610473.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [12]:
booster

NameError: name 'booster' is not defined